<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/azure_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%pip install -q azure-ai-ml omegaconf python-dotenv loguru textract tiktoken openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os,argparse,loguru, json, time, datetime, datetime
from pathlib import Path
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import AmlCompute
from loguru import logger

In [ ]:
cwd = Path.cwd()
json_env_path = Path(cwd, "azure.env")
json_env_path = json_env_path if json_env_path.is_file() else None;json_env_path

In [ ]:
def maybe_load_aml_env_vars(json_env_path=None):
  import os, json
  if not json_env_path: return False

  try:
    with open(json_env_path, "r") as f:
      env_vars = json.load(f)
    os.environ["RESOURCE_NAME"] = env_vars["RESOURCE_NAME"]
    os.environ["WORKSPACE_NAME"] = env_vars["WORKSPACE_NAME"]
    os.environ["SUBSCRIPTION_ID"] = env_vars["SUBSCRIPTION_ID"]

  except Exception as e:
    logger.error(f"{e}")

  return (os.getenv("RESOURCE_NAME")
          and os.getenv("WORKSPACE_NAME")
          and os.getenv("SUBSCRIPTION_ID")
          )

@logger.catch
def aml_env_connected():
  "use environ vars to connect to Azure ML"
  pass

@logger.catch
def maybe_start_experiment(AZURE_AVAILABLE):
  if AZURE_AVAILABLE:
    pass

@logger.catch
def maybe_stop_experiment(AZURE_AVAILABLE):
  if AZURE_AVAILABLE:
    pass


In [ ]:
maybe_load_aml_env_vars(json_env_path)

os.getenv("SUBSCRIPTION_ID")

'SUBSCRIPTION_ID'



## 1 OBTAIN *DATASET*




### extract text from pdf

In [13]:
import textract
import os
import openai
import tiktoken

In [26]:
# upload resumes to this path
resume_path = cwd / "Resumes"
resume_path.mkdir(exist_ok=True)

In [27]:
#TODO walk the directory to get all the filenames, use as names of the people in tagging and document retrieval
files = ["Jesse_Jayant.pdf", "Nadia_Smythe.pdf", "jin_wang.pdf" ,"sarah_jones.pdf"]

In [34]:
# Extract the raw text from each PDF using textract
# TODO Extract the file name and save it in a separate name field, or save these using name as dictkey, or save in dataframe row
texts =[textract.process((resume_path / file), method='pdfminer').decode('utf-8') for file in files]

clean_texts = [text.replace("  ", " ").replace("\n", "; ").replace(';',' ') for text in texts]

In [35]:
clean_texts[0]

'Jesse Jayant     555-555-5555/ you@post.harvard.edu    Summary     Results-oriented finance professional with over 10 years of experience in publicly traded and privately held   enterprises. Proven track record in complex and capital-intensive global industries, delivering value and innovation in   Finance, Strategy, and Corporate Planning.     •  •  •    Budgeting  Business Planning  Financial Modeling    Strategic Planning  •  Reporting  •  • Valuation    • Month-End Close  Capital Planning  •  Project Planning  •    Core Competencies     Financial Analysis  Forecasting    •  •  • M&A    Experience     USA Airlines, Chicago, IL         Sr. Financial Analyst, Information Technology Financial Planning      2011 - 2018     • Developed and monitored a $1B annual Information Technology (IT) budget for 2012 & 2013.  •  •  •    Tracked spending against budget and project progress to ensure effectiveness of financial controls and accuracy.  Prepared and presented monthly, quarterly, and ann

In [36]:
for i, text in enumerate(clean_texts):
  print(len(text)) #count characters

5357
2906
3078
5067


In [ ]:
for i, text in enumerate(clean_texts):
  print(len(text)) #count characters